# Changes description compared to 0.2
- Add more different benign traffic to imitate real world data 
- Set `persistent=0` in attack traffic to use separate connections for each attack instead of reusing the same connection

## Collecting data from Campus infrastructure

In [ ]:
import sys
sys.version

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import os

from netunicorn.base import MinionPool, Task
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.client.remote import RemoteClient

In [ ]:
experiment_label_1 = 'patator-0.3.14-1'
experiment_label_2 = 'patator-0.3.14-2'

In [ ]:
login = os.environ['VALID_VICTIM_LOGIN']
password = os.environ['VALID_VICTIM_PASSWORD']
class BenignTask(Task):
    def __init__(self, url):
        self.url = url
    
    def run(self):
        import subprocess
        for i in range(100):
            subprocess.check_output(f'curl ' + self.url, shell=True)
        return 0

class BenignTaskAsync(Task):
    def __init__(self, url):
        self.url = url
    
    def run(self):
        import aiohttp
        import asyncio
        import nest_asyncio
        nest_asyncio.apply()
        
        async def request():
            async with aiohttp.ClientSession() as session:
                r = await session.get(self.url)
                r.close()
        
        async def main():
            requests = [request() for _ in range(10)]
            await asyncio.gather(*requests)
        
        loop = asyncio.get_event_loop()
        for i in range(100):
            loop.run_until_complete(main())
        return 0

class BenignTaskAuthorized(Task):
    def __init__(self, label: str):
        self.label = label
    
    def run(self):
        import requests
        for i in range(100):
            requests.get('redacted/unicorn/mediator/api/v1/experiment/' + self.label, auth=(f'{login}', f'{password}'))
        return 0

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "persistent=0 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
benign_pipeline = Pipeline()
for i in range(20):
    (benign_pipeline
     .then(BenignTask('https://redacted/about/'))
     .then(BenignTask('https://redacted/unicorn/mediator/health'))
     .then(BenignTaskAsync('https://redacted/about/'))
     .then(BenignTaskAuthorized(experiment_label_1))
    )
    

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [ ]:
minion_pool = client.get_minion_pool()

In [ ]:
pool_1 = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:75:6b:2c',
    'raspi-e4:5f:01:72:a2:eb',
    'raspi-e4:5f:01:56:d8:fc',
}])

pool_2 = MinionPool([x for x in minion_pool if x.name in {
    'raspi-dc:a6:32:d7:6e:64',
    'raspi-e4:5f:01:56:d9:73',
    'raspi-e4:5f:01:56:d9:8b'
}])

In [ ]:
assert len(pool_1) == 3
assert len(pool_2) == 3

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)

## Collecting from Azure infrastructure

In [ ]:
experiment_label_1 = 'patator-0.3.8-azure-1'
experiment_label_2 = 'patator-0.3.8-azure-2'

In [ ]:
benign_pipeline = Pipeline()
for i in range(20):
    (benign_pipeline
     .then(BenignTask('http://172.174.8.19/about.html'))
     .then(BenignTask('http://172.174.8.19:26511/health'))
     .then(BenignTaskAsync('http://172.174.8.19/about.html'))
     .then(BenignTaskAuthorized(experiment_label_1))
    )
    

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=0 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)

## Collecting from multicloud infrastructure

In [ ]:
experiment_label_1 = 'patator-0.3.9-multicloud-1'
experiment_label_2 = 'patator-0.3.9-multicloud-2'

In [ ]:
benign_pipeline = Pipeline()
for i in range(20):
    (benign_pipeline
     .then(BenignTask('http://172.174.8.19/about.html'))
     .then(BenignTask('http://172.174.8.19:26511/health'))
     .then(BenignTaskAsync('http://172.174.8.19/about.html'))
     .then(BenignTaskAuthorized(experiment_label_1))
    )
    

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=0 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [ ]:
pool_1 = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-1',
    'netunicorn-search-digitalocean-1',
}])
pool_2 = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-2',
    'netunicorn-search-aws-3',
    'netunicorn-search-digitalocean-2',
}])

In [ ]:
assert len(pool_1) == 2
assert len(pool_2) == 3

In [ ]:
experiment_1 = Experiment().map(pool_1, attacker_pipeline).map(pool_2, benign_pipeline)
experiment_2 = Experiment().map(pool_2, attacker_pipeline).map(pool_1, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment_1, experiment_label_1)

In [ ]:
client.prepare_experiment(experiment_2, experiment_label_2)

In [ ]:
client.start_execution(experiment_label_1)

In [ ]:
client.start_execution(experiment_label_2)